In [ ]:
import cv2
import numpy as np
from IPython import display
import ultralytics
from ultralytics import YOLO
import threading
# from skimage.metrics import structural_similarity as ssim

'''
    Global variables useful to communicate between different threads:
        - frame2_resized: frame in which there will be a trace (hopefully)
        - results_ready: flag that indicates if the "YOLO thread" has finished its inference. 
                         This flag is useful to not open more than two thread (the main one, the YOLO one).
'''

frame2_resized = np.zeros((480, 640, 3), dtype=np.uint8) 
results_ready = True 

def analyze_frame_YOLO(frame1, trace_prob_threshold):
    
    ''' 
    The function analyzes one frame coming from the video with YOLOv8 model.
    It is thought to be called in a separated thread, so the function run in parallel respect to the main program.
    The function uses "threading" and "ultralytics" libraries to work.
    :param frame1: frame we want to analyze with YOLO algorithm.
    :return:
    '''
    
    # function that run in another thread doesn't have a "return" line, so we must declare global variables to communicate between threads and functions.
    global frame2_resized, results_ready
    
    # does the inference
    result = model(frame1, conf=0.25, task="classify", mode="predict", save=False, save_txt=False, save_conf=True, save_crop=False, verbose=False)
    
    # if the trace probability is greater than the probability threshold, then "return" the analyzed frame to the main program 
    trace_prob = result[0].probs.data[2].item()
    if trace_prob > trace_prob_threshold:
        frame2_resized = cv2.resize(frame1.copy(), (640, 480))

    # flag that indicates if the inference is over
    results_ready = True

def video(cap, trace_prob_threshold = 0.3, delta_frame = 1):
    
    '''
    The function shows the live video from webcam and the trace frames analyzed by YOLO algorithm (runned in parallel in another thread) 
    in two different boxes.
    The function uses "cv2" and "numpy" libraries to work. 
    :param cap: cap cv2-object from which the function takes frames.
    :param trace_prob_threshold: float from 0 to 1; threshold for selecting traces. Set to 0.3 by default.
    :param delta_frame: it is another way to control the running of YOLO algorithm. Selects every how many frames to analyze the current frame. Set to 1 by default.
    :return: 
    '''
    
    # function that run in another thread doesn't have a "return" line, so we must declare global variables to communicate between threads and functions.
    global frame2_resized, results_ready
    
    i_counter = delta_frame  # frame counter
    
    # white stripes to better visualize the video boxes.
    white_stripe1 = np.ones((480, 20, 3), dtype=np.uint8) * 255 
    white_stripe2 = np.ones((20, 1340, 3), dtype=np.uint8) * 255  
    
    while True:
    
        ret1, frame1 = cap.read()
    
        if not ret1:
            print('Errore nella lettura del frame dalla webcam')
            break
    
        # Resizes frame1
        frame1_resized = cv2.resize(frame1, (640, 480))
    
        # if the precedent YOLO run is concluded, then begins another run
        if results_ready is True and i_counter % delta_frame == 0:
            threading.Thread(target=analyze_frame_YOLO, args=(frame1,trace_prob_threshold)).start()
            '''
            if flag == True: pre_frame = frame2_resized
            if ssim(cv2.cvtColor(pre_frame, cv2.COLOR_RGB2GRAY), cv2.cvtColor(frame2_resized, cv2.COLOR_RGB2GRAY), full=False) < 0.5: 
                flag = True
            else:
                frame2_resized = pre_frame
            '''
            results_ready = False
            
        # combines all the images
        combined_frame_ = cv2.hconcat([white_stripe1, frame1_resized, white_stripe1, frame2_resized, white_stripe1])
        combined_frame = cv2.vconcat([combined_frame_, white_stripe2])
        
        cv2.imshow('Live Trigger', combined_frame)
    
        i_counter += 1
    
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    return

# checks computer
display.clear_output()
ultralytics.checks()

Ultralytics YOLOv8.2.76  Python-3.11.8 torch-2.4.0+cpu CPU (Intel Core(TM) i3-5010U 2.10GHz)
Setup complete  (4 CPUs, 11.9 GB RAM, 168.6/223.0 GB disk)


In [2]:
model = YOLO("YOLO-CC-V1.pt")

# Initializes the webcam (1 is the second computer webcam)
cap = cv2.VideoCapture(rf'C:\Users\Black\Downloads\20240904_165241_1.mp4')
# Checks if the webcam is opened properly
if not cap.isOpened():
    print(r"Errore nell'apertura della webcam")
    exit()

video(cap)

cap.release()
cv2.destroyAllWindows()